## Нужные в этом тяжёлом деле (и не очень) библиотеки и модули

In [4]:
import numpy as np
import random
from math import log, prod 
import simpy
import pandas
import scipy.stats
import statsmodels 

## Генерация случайной величины
Экспоненциальное распределение с вводимым параметром lambda

In [30]:
lambda_ = 6.0
k = 1
std = 0
random.seed(1111)
# Экспоненциальное
[print(f'{(-log(random.random()) / lambda_) - (1 / lambda_)}, \
    {random.expovariate(lambda_) - 1 / lambda_}') for _ in range(5)]
# print(random.expovariate(lambda_))
print()
# Эрланга
tmp = lambda_ / k 
print(-log(prod([random.random() for _ in range(k)])) / tmp)
# имеет два параметра: альфа (k) и бета
print(random.gammavariate(k, lambda_))
print()
# нормальное
lambda_ = 0
k = 1
std = 1
tmp = lambda_ + std * (sum([random.random() for _ in range(12)]) - 6)
print(tmp)
if tmp < 0: tmp = 0.001
print(random.normalvariate(lambda_, std))
print()
# постоянная величина
print(lambda_)

0.08751553621397293,     -0.09630415889285317
-0.09287133313818126,     -0.09120593558848397
0.24697356333172363,     -0.04977272474396176
0.11238752301460928,     0.1926848352550752
0.014013031340297022,     0.024908515084732297

0.13732829810413735
5.993473330779972

-0.20833791121167078
-1.6180189004839174

0


## Простой процесс на simpy

In [7]:
def car(env):
    while True:
        print(f'Start parking at {env.now}')
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f'Start driving at {env.now}')
        trip_duration = 2
        yield env.timeout(trip_duration)

env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


## Запуск подпроцесса charge после парковки

In [8]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.run(until=15)

3042991530496 Start parking and charging at 0
3042991530448 Start parking and charging at 0
3042991530496 Start driving at 5
3042991530448 Start driving at 5
3042991530496 Start parking and charging at 7
3042991530448 Start parking and charging at 7
3042991530496 Start driving at 12
3042991530448 Start driving at 12
3042991530496 Start parking and charging at 14
3042991530448 Start parking and charging at 14


## Прерывание другого процесса

In [9]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{id(self)} Start parking and charging at {self.env.now}')
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print(f'{id(self)} Was interrupted. Hope, the battery is full enough ...')

            # The charge process has finished and
            # we can start driving again.
            print(f'{id(self)} Start driving at {self.env.now}')
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)


def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

env = simpy.Environment()
car_1 = Car(env)
car_2 = Car(env)
env.process(driver(env, car_1))
env.run(until=15)

3042991526752 Start parking and charging at 0
3042991529104 Start parking and charging at 0
3042991526752 Was interrupted. Hope, the battery is full enough ...
3042991526752 Start driving at 3
3042991526752 Start parking and charging at 5
3042991529104 Start driving at 5
3042991529104 Start parking and charging at 7
3042991526752 Start driving at 10
3042991526752 Start parking and charging at 12
3042991529104 Start driving at 12
3042991529104 Start parking and charging at 14


## Использование ресурсов
Добавляется устройство обслуживания с ёмкостью 2.
Машины 1 и 2 начинают обслуживаться сразу же по прибытии, а 3 и 4 ожидают освобождения слотов для обслуживания.

In [10]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    # Request one of its charging spots
    print(f'{name} arriving at {env.now}')
    with bcs.request() as req:
        yield req
        # Charge the battery
        print(f'{name} starting to charge at {env.now}')
        yield env.timeout(charge_duration)
        print(f'{name} leaving the bcs at {env.now}')

env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range(4):
    env.process(car(env, f'Car {i+1}', bcs, i*2, 5))
env.run()

Car 1 arriving at 0
Car 1 starting to charge at 0
Car 2 arriving at 2
Car 2 starting to charge at 2
Car 3 arriving at 4
Car 1 leaving the bcs at 5
Car 3 starting to charge at 5
Car 4 arriving at 6
Car 2 leaving the bcs at 7
Car 4 starting to charge at 7
Car 3 leaving the bcs at 10
Car 4 leaving the bcs at 12


## Пример с банком
В банке есть сотрудники, обслуживающие клиентов, и терминалы, выдающие талоны. Есть, соответственно два процесса: выдача талона и обслуживание клиента. 

In [11]:
class Bank(object):
    def __init__(self, env, num_employee, num_terminal):
        self.env = env
        self.employee = simpy.Resource(env, num_employee)
        self.terminal = simpy.Resource(env, num_terminal)

    def service(self, customer):
    # Обслуживание занимает время в интервале от 1 до 15 минут
        yield self.env.timeout(rd.randint(1, 15))

    def take_token(self, customer):
    # Получение талона занимает до 45 секунд
        yield self.env.timeout(45/60)

Создадим функцию, которая будет отвечать за поведение клиента в среде:

In [12]:
def go_to_bank(env, customer, bank):
	# Клиент пришел в банк
	arrival_time = env.now
	
	with bank.terminal.request() as request:
		yield request
		yield env.process(bank.take_token(customer))

	with bank.employee.request() as request:
		yield request
		yield env.process(bank.service(customer))

	global times
	times.append(env.now - arrival_time)

Теперь необходимо создать функцию для запуска моделирования. Она будет отвечать за создание экземпляра банка и генерацию клиентов, до тех пор, пока симуляция не остановится:

In [13]:
def run_bank(env, num_employee, num_terminal):
	bank = Bank(env, num_employee, num_terminal)
	customer = 1
	
	env.process(go_to_bank(env, customer, bank))
	
	while True:
		# Предположим каждые 4 минуты заходит новый клиент
		yield env.timeout(rd.expovariate(1.0 / 240))
		customer += 1
		env.process(go_to_bank(env, customer, bank))

Отдельно создадим функцию для подсчета среднего времени:

In [14]:
def get_average_time(times):
	average_time = np.mean(times)
	minutes, frac_minutes = divmod(average_time, 1)
	seconds = frac_minutes * 60
	return round(minutes), round(seconds)

Запустим моделирование:

In [15]:
rd.seed(42)
# Начальные данные
num_employee = 7 # В офисе работает 7 сотрудников
num_terminal = 1 # В офисе 1 терминал по выдаче талонов
times = []

# Запуск моделирования
env = simpy.Environment()
env.process(run_bank(env, num_employee, num_terminal))
env.run(until=72)

# Результаты
mins, secs = get_average_time(times)
print(f'\nСреднее время обслуживания: {mins}:{secs}')

NameError: name 'rd' is not defined

## СМО M/M/1

In [22]:
class System(object):
    def __init__(self, env, num_services, mu):
        self.env = env
        self.server = simpy.Resource(env, num_services)
        self.mu = mu

    def service(self, demand):
        # yield self.env.timeout(rd.expovariate(self.mu))
        yield self.env.timeout(env.now - log(random.random()) / mu)
        global cnt_demands
        cnt_demands.append(self.server.count - 1)
        print(f'Требование {demand} обслужено в {self.env.now}')

In [23]:
def go_to_system(env, demand, system):
	# требование пришло в систему
	arrival_time = env.now
	
	with system.server.request() as request:
		yield request
		yield env.process(system.service(demand))

	global times
	times.append(env.now - arrival_time)

In [24]:
def generate(env, lambda_, demand):
    try:
        yield env.timeout(env.now - log(random.random()) / lambda_)
        demand += 1
        print(f'Требование {demand} поступило в {env.now}')
    except simpy.Interrupt:
        print(f'Генерирование прервано в {env.now}')

In [27]:
def run_system(env, num_services, lambda_, mu):
	system = System(env, num_services, mu)
	demand = 1
	print(f'Требование {demand} поступило в {env.now}')
	env.process(go_to_system(env, demand, system))
	global cnt_demands
	cnt_demands.append(system.server.count + 1)
	
	while True:
		generator = env.process(generate(env, lambda_, demand))
		generator_acivate = env.event()
		cnt_demands.append(system.server.count + 1)
		server = env.process(go_to_system(env, demand, system))
		server_acivate = env.event()

In [28]:
cnt = 1
all_times = []
all_demands = []
for _ in range(cnt):
    # Начальные данные
    num_services = 2 # число приборов
    lambda_ = 1
    mu = 2
    times = []
    cnt_demands = []

    # Запуск моделирования
    env = simpy.Environment()
    env.process(run_system(env, num_services, lambda_, mu))
    env.run(until=10**6)

    # Результаты
    average_time = np.mean(times)
    print(f'\nСреднее время обслуживания: {average_time}')
    average_cnt = np.mean(cnt_demands)
    print(cnt_demands)
    print(f'Число требований в системе: {average_cnt}')
    all_times.append(average_time)
    all_demands.append(average_cnt)
print(np.mean(all_times))
print(np.mean(all_demands))

Требование 1 поступило в 0


KeyboardInterrupt: 

In [ ]:
class First:
    def __init__(self, id):
        self.id = id

first = First(1)
second = First(2)

print(first.id, second.id, '\n', id(first), id(second))

first = First(3)
print(first.id, second.id, '\n', id(first), id(second))

1 2 
 1555982079536 1555982089760
3 2 
 1556264903840 1555982089760


In [ ]:
f = open('out.txt', 'a')
f.write('abc')
f.close()
f = open('out.txt', 'a')
f.write('I ate your string')
f.close()

In [ ]:
f = open('test.txt', 'w')
f.write('some shit happens')
f.close()
f = open('test.txt', 'r')
for line in f:
    print(line)
f.close()

f = open('test.txt', 'a')
f.write('\nbut we must stand')
f.close()
f = open('test.txt', 'r')
for line in f:
    print(line)
f.close()

some shit happens
some shit happens

but we must stand


In [ ]:
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec())

NameError: name 'QtWidgets' is not defined

In [ ]:
a = [1, 2, 3, 4]
a[:2]

[1, 2]

In [31]:
a = [1, 2, 6, 9]
for item in a:
    if item == 6:
        ind = a.index(item)
print(f'dsfg {a[ind]} sdfsd {ind}')

dsfg 6 sdfsd 2
